In [1]:
"import all what we need for this projet"
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute,Aer,IBMQ,assemble
from numpy import pi
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.compiler import transpile
import warnings
warnings.filterwarnings('ignore')

In [2]:
def Alice_basis(p,b) -> QuantumCircuit: 
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    qca = QuantumCircuit(qreg_q, creg_c)
    qcbase = QuantumCircuit(qreg_q, creg_c)
    "alice code the qubit with random bases"
    if (b=="ht"):
        qcbase.h(qreg_q[int(p)])
        qcbase.t(qreg_q[int(p)])
    if (b=="hs"):
        qcbase.h(qreg_q[int(p)])
        qcbase.s(qreg_q[int(p)])    
    if (b=="hz"):
        qcbase.h(qreg_q[int(p)])
        qcbase.z(qreg_q[int(p)])
    if (b=="h"):
        qcbase.h(qreg_q[int(p)])
    return qcbase
def Alice_Circuit():
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    qca = QuantumCircuit(qreg_q, creg_c)
    qcbase = QuantumCircuit(qreg_q, creg_c)

    for m in range(5):
        if str("01101")[m]=="1":
            #alice initialize the qubit with 0 or 1 in the begining(in the interface created)
            qca.x(qreg_q[m])
    qca.barrier(qreg_q[3], qreg_q[2], qreg_q[4], qreg_q[1], qreg_q[0])
    print("***Alice encode phase***")
    for i in range(5):
        p = input("print the position of the encode basis:\t") 
        b = input("print the encode basis:\t") 
        qcbase=Alice_basis(p,b)
        qca+=qcbase
    "between this two barriers named quantum channel that alice and bob can exchange qubit "
    qca.barrier(qreg_q[3], qreg_q[2], qreg_q[4], qreg_q[1], qreg_q[0])
    
    print(qca)
    return(qca)

def Bob_basis(p,b) -> QuantumCircuit:   
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    qca = QuantumCircuit(qreg_q, creg_c)
    qcbase = QuantumCircuit(qreg_q, creg_c)
    "alice code the qubit with random bases"
    if (b=="th"):
        qcbase.tdg(qreg_q[int(p)])
        qcbase.h(qreg_q[int(p)])
        
    if (b=="sh"):
        qcbase.sdg(qreg_q[int(p)])
        qcbase.h(qreg_q[int(p)])
            
    if (b=="zh"):
        qcbase.z(qreg_q[int(p)])
        qcbase.h(qreg_q[int(p)])
        
    if (b=="h"):
        qcbase.h(qreg_q[int(p)])
    return qcbase
def Bob_Circuit():
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    qcb = QuantumCircuit(qreg_q, creg_c)
    qcbase = QuantumCircuit(qreg_q, creg_c)

    print("***Bob decode phase***")
    for i in range(5):
        p = input("print the position of the decode basis:\t") 
        b = input("print the decode basis:\t") 
        qcbase=Bob_basis(p,b)
        qcb+=qcbase
    "between this two barriers named quantum channel that alice and bob can exchange qubit "
    qcb.barrier(qreg_q[3], qreg_q[2], qreg_q[4], qreg_q[1], qreg_q[0])
    print(qcb)
    return(qcb)

def Eve_Circuit(Ke):
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    qce = QuantumCircuit(qreg_q, creg_c)
    qcbase = QuantumCircuit(qreg_q, creg_c)
    str1 = ', '.join(str(e) for e in Ke)
    print("***Eve intercept phase, on Qubit index "+str1+" ***")
        
    print("***Eve decode***")
  
    for i in range(len(Ke)):
        b = input("print the decode basis:\t") 
        
        qcbase=Bob_basis(str(Ke[i]),b)
        qce+=qcbase
    
    for i in Ke:
        qce.measure(qreg_q[int(i)], creg_c[int(i)])
    print("***Eve encode***")
    
    for i in range(len(Ke)):
        b = input("print the encode basis:\t") 
        
        qcbase=Alice_basis(str(Ke[i]),b)
        qce+=qcbase
    qce.barrier(qreg_q[3], qreg_q[2], qreg_q[4], qreg_q[1], qreg_q[0])
    print(qce)
    return(qce)

In [3]:
"Creation of the first circuit ( the communication between alice and bob without eve)"
def BB84_without_eve(Ka):
    "------------------------------------------------"
    "crate our circuit to exchange the information between alice and bob"
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    "------------------------------------------------"
    circuit = Alice_Circuit()
    "--------------------------------------------------------------------------------"
    circuit.barrier(qreg_q[3], qreg_q[2], qreg_q[4], qreg_q[1], qreg_q[0])
    "bob decode the qubit or the information with random bases"
    circuit = circuit + Bob_Circuit()
    
    "bob measure the qubit after the decode to have him unique key"
    circuit.measure(qreg_q[0], creg_c[0])
    circuit.measure(qreg_q[1], creg_c[1])
    circuit.measure(qreg_q[2], creg_c[2])
    circuit.measure(qreg_q[3], creg_c[3])
    circuit.measure(qreg_q[4], creg_c[4])
    "-----------------------------------------------------------------------------------"
    
    aer_sim = Aer.get_backend('aer_simulator')
    qobj = assemble(circuit, shots=8192)
    job = aer_sim.run(qobj)
    
    #list of bob key that he can use
    hist = job.result().get_counts() 

    max_val = list(hist.values())
    max_ke = list(hist.keys())
    #bob choose the highest probability key that he can uses securly
    Kb=str(max_ke[max_val.index(max(max_val))])
    text=""
    for i in [0,1,4]: #[0,1,4] is the index of the simelar bit can Alice and Bob exchange
        
        text+=Kb[i] #the final secure key that alice and bob can use
        
    "return the bob's key and the secure key"
    return("bob's key :\n"+Kb+"\n\n"+"the final generate key is:\n"+text)


In [4]:
"Creation of the first circuit ( the communication between alice and bob with eve)"
def BB84_with_eve(Ka,Ke):
    qreg_q = QuantumRegister(5, 'q')
    creg_c = ClassicalRegister(5, 'c')
    "-------------------------------------------------------------------------"
    "sort the eve key to impliment the attack on the quantum chanel "
    p=[]
    for m in range(len(Ke)):
        p+=[Ke[m]]
    p.sort()
    Ke=p
    "--------------------------------------------------------------------------"
    qca = Alice_Circuit()

    "-------------------------------------------------------------------------"
    "in this part(the third part in the circuit) eve intercept the quantum channel"
    qce=Eve_Circuit(Ke)
    "-------------------------------------------------------------------------"
    qcb=Bob_Circuit()
    
    qcb.measure(qreg_q[0], creg_c[0])
    qcb.measure(qreg_q[1], creg_c[1])
    qcb.measure(qreg_q[2], creg_c[2])
    qcb.measure(qreg_q[3], creg_c[3])
    qcb.measure(qreg_q[4], creg_c[4])

    qc=qca+qce+qcb
    aer_sim = Aer.get_backend('aer_simulator')
    qobj = assemble(qc, shots=8192)
    job = aer_sim.run(qobj)
    hist = job.result().get_counts()
    
    max_val = list(hist.values())
    max_ke = list(hist.keys())

    Kb=str(max_ke[max_val.index(max(max_val))])
    text=""
    for i in [0,1,4]:
        text+=Kb[i] 

    return("bob's key :\n"+Kb+"\n\n"+"the final generate key is:\n"+text)



In [5]:
import tkinter as tk
from PIL import ImageTk, Image

frame = tk.Tk()
frame.title("BB84 M.A.Garrach")
frame.geometry('500x350')
"--------------------------------------------------------------------------"
def printInput():
	inp = inputtxt.get(1.0, "end-1c")
	inp1 = inputtxt1.get(1.0, "end-1c")
	if (inp.isnumeric() and inp1.isnumeric() and len(inp)==5 and len(inp1)<=5):
		lbl1.config(text = BB84_with_eve(inp,inp1))
	else:
		lbl1.config(text="Incorrect!!\n *Alice's key should be \nbinary of length 5.\n*Eve should attack numbers \nbetween 0 and 4 and not repeat.")

def printInput1():
	inp = inputtxt.get(1.0, "end-1c")
	if inp.isnumeric() and len(inp)==5:
		lbl.config(text = BB84_without_eve(inp))
	else:
		lbl.config(text="Incorrect!!\n *Alice's key should be \nbinary of length 5.")
"--------------------------------------------------------------------------"
# TextBox Creation
inputtxt = tk.Text(frame,
				height = 1,
				width = 25)

inputtxt.pack()
inputtxt.place(x=150,y=90)

inputtxt1 = tk.Text(frame,
				height = 1,
				width = 25)

inputtxt1.pack()
inputtxt1.place(x=150,y=140)

# Button Creation
printButton = tk.Button(frame,
						text = "secure key",
						command = printInput)

printButton.pack()
printButton.place(x=350,y=210)

printButton1 = tk.Button(frame,
						text = "secure key",
						command = printInput1)
printButton1.pack()
printButton1.place(x=100,y=210)

# Label Creation
lbl = tk.Label(frame, text = "")
lbl.pack()
lbl.place(x=70,y=250)

lbl1 = tk.Label(frame, text = "")
lbl1.pack() 
lbl1.place(x=320,y=250)

#text Creation
text = tk.Label(frame, text = "BB84 implementation ",font=("Matura MT Script Capitals", 19),fg='blue')
text.pack()
text.place(x=120,y=15)

text0 = tk.Label(frame, text = "Alice's key :")
text0.pack()
text0.place(x=150,y=65)

text1 = tk.Label(frame, text = "Generate key with Eve",fg='red')
text1.pack()
text1.place(x=320,y=180)

text2 = tk.Label(frame, text = "Generate key without Eve",fg='green')
text2.pack()
text2.place(x=60,y=180)

text3 = tk.Label(frame, text = "Eve's Attack :")
text3.pack()
text3.place(x=150,y=115)

#import images
img= (Image.open("Desktop/alice.png"))
#Resize the Image using resize method
resized_image= img.resize((40,40), Image.ANTIALIAS)
new_image= ImageTk.PhotoImage(resized_image)
panel = tk.Label(frame, image = new_image)
panel.pack(side = "bottom", fill = "both", expand = "yes")
panel.place(x=100,y=70)

img1= (Image.open("Desktop/eve.png"))
#Resize the Image using resize method
resized_image1= img1.resize((45,45), Image.ANTIALIAS)
new_image1= ImageTk.PhotoImage(resized_image1)
panel = tk.Label(frame, image = new_image1)
panel.pack(side = "bottom", fill = "both", expand = "yes")
panel.place(x=100,y=120)

img2= (Image.open("Desktop/bob.png"))
#Resize the Image using resize method
resized_image2= img2.resize((40,40), Image.ANTIALIAS)
new_image2= ImageTk.PhotoImage(resized_image2)
panel = tk.Label(frame, image = new_image2)
panel.pack(side = "bottom", fill = "both", expand = "yes")
panel.place(x=240,y=240)

img3= (Image.open("Desktop/key.gif"))
#Resize the Image using resize method
resized_image3= img3.resize((40,20), Image.ANTIALIAS)
new_image3= ImageTk.PhotoImage(resized_image3)
panel = tk.Label(frame, image = new_image3)
panel.pack(side = "bottom", fill = "both", expand = "yes")
panel.place(x=240,y=300)

frame.mainloop()

***Alice encode phase***
print the position of the encode basis:	0
print the encode basis:	ht
print the position of the encode basis:	1
print the encode basis:	hs
print the position of the encode basis:	2
print the encode basis:	hs
print the position of the encode basis:	3
print the encode basis:	hz
print the position of the encode basis:	4
print the encode basis:	ht
           ░ ┌───┐┌───┐ ░ 
q_0: ──────░─┤ H ├┤ T ├─░─
     ┌───┐ ░ ├───┤├───┤ ░ 
q_1: ┤ X ├─░─┤ H ├┤ S ├─░─
     ├───┤ ░ ├───┤├───┤ ░ 
q_2: ┤ X ├─░─┤ H ├┤ S ├─░─
     └───┘ ░ ├───┤├───┤ ░ 
q_3: ──────░─┤ H ├┤ Z ├─░─
     ┌───┐ ░ ├───┤├───┤ ░ 
q_4: ┤ X ├─░─┤ H ├┤ T ├─░─
     └───┘ ░ └───┘└───┘ ░ 
c: 5/═════════════════════
                          
***Eve intercept phase, on Qubit index 1, 3, 4 ***
***Eve decode***
print the decode basis:	th
print the decode basis:	zh
print the decode basis:	th
***Eve encode***
print the encode basis:	ht
print the encode basis:	hz
print the encode basis:	ht
                                